In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np


df = pd.read_csv("/content/Airlines.csv")

# Preprocessing
df['resolutionRate'] = (df['closedGrievancesWithoutEscalation'] + df['closedGrievancesWithEscalation']) / df['totalReceived']
df['escalationRate'] = (df['activeGrievancesWithEscalation'] + df['closedGrievancesWithEscalation']) / df['totalReceived']
df['totalResolved'] = df['grievancesWithFeedbackIssueResolved']
df['totalSocialMediaGrievances'] = df['twitterGrievances'] + df['facebookGrievances']
df['offlineGrievances'] = df['totalReceived'] - df['totalSocialMediaGrievances']

df['resolvedViaSocial'] = (df['totalSocialMediaGrievances'] / df['totalReceived']) * df['totalResolved']
df['resolvedViaOffline'] = (df['offlineGrievances'] / df['totalReceived']) * df['totalResolved']


rating_columns = [
    'grievancesWithVeryGoodRating',
    'grievanceswithgoodrating',
    'grievancesWithOKRating',
    'grievancesWithBadRating',
    'grievancesWithVeryBadRating'
]

# Grouping data
top_grievances = df.groupby('subcategory')['totalReceived'].sum().sort_values(ascending=False)
customer_satisfaction = df.groupby('subcategory')[rating_columns].sum()
type_issues = df.groupby('type')['totalReceived'].sum().sort_values(ascending=False)
social_media = df.groupby('subcategory')[['twitterGrievances', 'facebookGrievances']].sum()
feedback_resolution = df.groupby('subcategory')[['grievancesWithFeedbackIssueResolved', 'grievancesWithFeedbackIssueNotResolved']].sum()
ratings_vs_resolution = df[['grievancesWithRatings', 'grievancesWithFeedbackIssueResolved', 'grievancesWithFeedbackIssueNotResolved']].corr()


sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)

def save_plot(fig, filename):
    fig.tight_layout()
    fig.savefig(filename)
    plt.close(fig)

# 1. Top Grievance Airlines
fig, ax = plt.subplots()
top_grievances.head(5).plot.pie(autopct='%1.1f%%', startangle=90, ax=ax)
ax.set_ylabel('')
ax.set_title("Grievance Share by Top 5 Airlines")
save_plot(fig, "grievance_share_pie.png")



# 2. Customer Satisfaction
top_rated_airlines = customer_satisfaction.sum(axis=1).sort_values(ascending=False).head(5).index
fig, ax = plt.subplots()
customer_satisfaction.loc[top_rated_airlines].T.plot(kind='bar', ax=ax)  # Transpose for grouped bars
ax.set_title("Customer Satisfaction (Top 5 Airlines)")
ax.set_ylabel("Satisfaction Score")
ax.set_xlabel("Satisfaction Criteria")
ax.legend(title="Airlines")
save_plot(fig, "customer_satisfaction_grouped.png")


# 3. Problematic Grievance Types
fig, ax = plt.subplots()
type_issues.head(10).plot(kind='barh', color='red', ax=ax)
ax.set_title("Top Grievance Types")
save_plot(fig, "grievance_types_barh.png")

# 4. Social Media Complaints
top_social = social_media.sum(axis=1).sort_values(ascending=False).head(5).index
fig, ax = plt.subplots()
social_media.loc[top_social].plot(kind='bar', ax=ax)
ax.set_title("Social Media Complaints (Top 5 Airlines)")
save_plot(fig, "social_media_grouped.png")

# 5. Social Media vs Offline Complaint Resolutions - BAR ok

airline_logged_resolved = df.groupby('subcategory').agg({
    'totalSocialMediaGrievances': 'sum',
    'offlineGrievances': 'sum',
    'grievancesWithFeedbackIssueResolved': 'sum'
})

airline_logged_resolved['Logged'] = airline_logged_resolved['totalSocialMediaGrievances'] + airline_logged_resolved['offlineGrievances']
airline_logged_resolved['Resolved'] = airline_logged_resolved['grievancesWithFeedbackIssueResolved']
airline_logged_resolved = airline_logged_resolved[['Logged', 'Resolved']]
top_airlines = airline_logged_resolved.sort_values(by='Logged', ascending=False).head(10)
x = np.arange(len(top_airlines))
width = 0.35

fig, ax = plt.subplots()
bars1 = ax.bar(x - width/2, top_airlines['Logged'], width, label='Logged', color='slateblue')
bars2 = ax.bar(x + width/2, top_airlines['Resolved'], width, label='Resolved', color='lightcoral')

ax.set_ylabel('Number of Complaints')
ax.set_title('Logged vs Resolved Complaints by Airline')
ax.set_xticks(x)
ax.set_xticklabels(top_airlines.index, rotation=45, ha='right')
ax.legend()
save_plot(fig, "airline_logged_vs_resolved_bar.png")


# 6. Feedback Resolution
total_feedback = feedback_resolution.sum()
fig, ax = plt.subplots()
total_feedback.plot.pie(autopct='%1.1f%%', startangle=90, ax=ax, colors=["skyblue", "salmon"])
ax.set_ylabel('')
ax.set_title("Overall Feedback Resolution")
save_plot(fig, "feedback_resolution_pie.png")


# 7. Ratings vs Resolutions
fig, ax = plt.subplots()
sns.heatmap(ratings_vs_resolution, annot=True, cmap='coolwarm', ax=ax)
ax.set_title("Ratings vs Feedback Resolution Correlation")
save_plot(fig, "ratings_resolution_heatmap.png")


